# Imports


In [1]:
import pandas as pd
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
export_dir = os.getcwd()
from pathlib import Path
import pickle
from collections import defaultdict
import time
import torch
import torch.nn as nn
import copy
import torch.nn.functional as F
import optuna
import logging
import matplotlib.pyplot as plt
import ipynb
import importlib

In [2]:
data_name = "ML1M" ### Can be ML1M, ML1M_demographic, Yahoo, Pinterest
recommender_name = "MLP" ### Can be MLP, VAE, MLP_model, GMF_model, NCF
DP_DIR = Path("processed_data", data_name) 
export_dir = Path(os.getcwd())
files_path = Path(export_dir, DP_DIR)
checkpoints_path = Path(export_dir, "checkpoints")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
output_type_dict = {
    "VAE":"multiple",
    "MLP":"single",
    "NCF": "single",
    "MLP_model": "single",
    "GMF_model": "single"
}

num_users_dict = {
    "ML1M":6037,
    "ML1M_demographic":6037,
    "Yahoo":13797, 
    "Pinterest":19155
}

num_items_dict = {
    "ML1M":3381,
    "ML1M_demographic":3381,
    "Yahoo":4604, 
    "Pinterest":9362
}

demographic_dict = {
    "ML1M_demographic": True,
    "ML1M":False,
    "Yahoo":False, 
    "Pinterest":False
}

features_dict = {
    "ML1M_demographic": 3421,
    "ML1M":None,
    "Yahoo":None, 
    "Pinterest":None
}

recommender_path_dict = {
    ("ML1M","VAE"): Path(checkpoints_path, "VAE_ML1M_0.0007_128_10.pt"),
    ("ML1M","MLP"):Path(checkpoints_path, "MLP1_ML1M_0.0076_256_7.pt"),
    ("ML1M","MLP_model"):Path(checkpoints_path, "MLP_model_ML1M_0.0001_64_27.pt"),
    ("ML1M","GMF_model"): Path(checkpoints_path, "GMF_best_ML1M_0.0001_32_17.pt"),
    ("ML1M","NCF"):Path(checkpoints_path, "NCF_ML1M_5e-05_64_16.pt"),

    ("ML1M_demographic","VAE"): Path(checkpoints_path, "VAE_ML1M_demographic_0.0001_64_6_18.pt"),
    ("ML1M_demographic","MLP"):Path(checkpoints_path, "MLP_ML1M_demographic_0.0_64_0_28.pt"),
    ("ML1M_demographic","MLP_model"):Path(checkpoints_path, "MLP_model2_ML1M_demographic_7e-05_128_3_22.pt"),
    ("ML1M_demographic","GMF_model"): Path(checkpoints_path, "GMF_model_ML1M_demographic_0.00061_64_21_10.pt"),
    ("ML1M_demographic","NCF"):Path(checkpoints_path, "NCF_ML1M_demographic_0.00023_32_3_2.pt"),
    
    ("Yahoo","VAE"): Path(checkpoints_path, "VAE_Yahoo_0.0001_128_13.pt"),
    ("Yahoo","MLP"):Path(checkpoints_path, "MLP2_Yahoo_0.0083_128_1.pt"),
    ("Yahoo","MLP_model"):Path(checkpoints_path, "MLP_model_Yahoo_5e-05_32_29.pt"),
    ("Yahoo","GMF_model"): Path(checkpoints_path, "GMF_model2_Yahoo_0.0_128_0_49.pt"),
    ("Yahoo","NCF"):Path(checkpoints_path, "NCF_Yahoo_0.001_64_21_0.pt"),
    
    ("Pinterest","VAE"): Path(checkpoints_path, "VAE_Pinterest_0.0002_32_12.pt"),
    ("Pinterest","MLP"):Path(checkpoints_path, "MLP_Pinterest_0.0062_512_21_0.pt"),
    ("Pinterest","MLP_model"):Path(checkpoints_path, "MLP_model_Pinterest_4e-05_1024_7_18.pt"),
    ("Pinterest","GMF_model"): Path(checkpoints_path, "GMF_model_Pinterest_0.001_512_3_19.pt"),
    ("Pinterest","NCF"):Path(checkpoints_path, "NCF2_Pinterest_9e-05_32_9_10.pt"),
    
}

hidden_dim_dict = {
    ("ML1M","VAE"): None,
    ("ML1M","MLP"): 32,
    ("ML1M","MLP_model"): 8,
    ("ML1M","GMF_model"): 8,
    ("ML1M","NCF"): 8,

    ("ML1M_demographic","VAE"): None,
    ("ML1M_demographic","MLP"): 32,
    ("ML1M_demographic","MLP_model"): 8,
    ("ML1M_demographic","GMF_model"): 8,
    ("ML1M_demographic","NCF"): 8,
    
    ("Yahoo","VAE"): None,
    ("Yahoo","MLP"):32,
    ("Yahoo","MLP_model"): 8,
    ("Yahoo","GMF_model"): 8,
    ("Yahoo","NCF"):8,
    
    ("Pinterest","VAE"): None,
    ("Pinterest","MLP"):512,
    ("Pinterest","MLP_model"): 64,
    ("Pinterest","GMF_model"): 64,
    ("Pinterest","NCF"): 64,
}

LXR_checkpoint_dict = {
    ("ML1M","VAE"): ('LXR_ML1M_VAE_16_19.pt',128),
    ("ML1M","MLP"): ('LXR_ML1M_MLP_9_20_1_7.143470844191349_0.35140294238438674.pt',128),
    'another': 'LXR_ML1M_MLP_16_20_1_4.962869318313412_0.01958043494301176,pt',
    ("ML1M","MLP_model"): 8,
    ("ML1M","GMF_model"): 8,
    ("ML1M","NCF"): 8,

    ("ML1M_demographic","VAE"): None,
    ("ML1M_demographic","MLP"): 32,
    ("ML1M_demographic","MLP_model"): 8,
    ("ML1M_demographic","GMF_model"): 8,
    ("ML1M_demographic","NCF"): 8,
    
    ("Yahoo","VAE"): None,
    ("Yahoo","MLP"):32,
    ("Yahoo","MLP_model"): 8,
    ("Yahoo","GMF_model"): 8,
    ("Yahoo","NCF"):8,
    
    ("Pinterest","VAE"): None,
    ("Pinterest","MLP"):512,
    ("Pinterest","MLP_model"): 64,
    ("Pinterest","GMF_model"): 64,
    ("Pinterest","NCF"): 64,
}

In [4]:
output_type = output_type_dict[recommender_name] ### Can be single, multiple
num_users = num_users_dict[data_name] 
num_items = num_items_dict[data_name] 
demographic = demographic_dict[data_name]
if demographic:
    num_features = features_dict[data_name]
else:
    num_features = num_items_dict[data_name]
hidden_dim = hidden_dim_dict[(data_name,recommender_name)]

recommender_path = recommender_path_dict[(data_name,recommender_name)]
lxr_path = LXR_checkpoint_dict[(data_name,recommender_name)][0]
lxr_dim = LXR_checkpoint_dict[(data_name,recommender_name)][1]

## Data and baselines imports

In [5]:
train_data = pd.read_csv(Path(files_path,f'train_data_{data_name}.csv'), index_col=0)
test_data = pd.read_csv(Path(files_path,f'test_data_{data_name}.csv'), index_col=0)
static_test_data = pd.read_csv(Path(files_path,f'static_test_data_{data_name}.csv'), index_col=0)
with open(Path(files_path,f'pop_dict_{data_name}.pkl'), 'rb') as f:
    pop_dict = pickle.load(f)
train_array = train_data.to_numpy()
test_array = test_data.to_numpy()
items_array = np.eye(num_items)
all_items_tensor = torch.Tensor(items_array).to(device)

In [6]:
test_array = static_test_data.iloc[:,:-2].to_numpy()

In [7]:
with open(Path(files_path, f'jaccard_based_sim_{data_name}.pkl'), 'rb') as f:
    jaccard_dict = pickle.load(f) 

In [8]:
with open(Path(files_path, f'cosine_based_sim_{data_name}.pkl'), 'rb') as f:
    cosine_dict = pickle.load(f) 

In [9]:
with open(Path(files_path, f'pop_dict_{data_name}.pkl'), 'rb') as f:
    pop_dict = pickle.load(f) 

In [10]:
with open(Path(files_path, f'tf_idf_dict_{data_name}.pkl'), 'rb') as f:
    tf_idf_dict = pickle.load(f) 

In [11]:
for i in range(num_features):
    for j in range(i, num_features):
        jaccard_dict[(j,i)]= jaccard_dict[(i,j)]
        cosine_dict[(j,i)]= cosine_dict[(i,j)]

In [12]:
pop_array = np.zeros(len(pop_dict))
for key, value in pop_dict.items():
    pop_array[key] = value

In [13]:
kw_dict = {'device':device,
          'num_items': num_items,
          'demographic':demographic,
          'num_features':num_features,
          'pop_array':pop_array,
          'all_items_tensor':all_items_tensor,
          'static_test_data':static_test_data,
          'items_array':items_array,
          'output_type':output_type,
          'recommender_name':recommender_name}

# Recommenders Architecture

In [14]:
from ipynb.fs.defs.recommenders_architecture import *
importlib.reload(ipynb.fs.defs.recommenders_architecture)
from ipynb.fs.defs.recommenders_architecture import *

VAE_config= {
"enc_dims": [512,128],
"dropout": 0.5,
"anneal_cap": 0.2,
"total_anneal_steps": 200000
}

In [15]:
def load_recommender():
    if recommender_name=='MLP':
        recommender = MLP(hidden_dim, **kw_dict)
    elif recommender_name=='VAE':
        recommender = VAE(VAE_config, **kw_dict)
    elif recommender_name=='MLP_model':
        recommender = MLP_model(hidden_size=hidden_dim, num_layers=3, **kw_dict)
    elif recommender_name=='GMF_model':
        recommender = GMF_model(hidden_size=hidden_dim, **kw_dict)
    elif recommender_name=='NCF':
        MLP_temp = MLP_model(hidden_size=hidden_dim, num_layers=3, **kw_dict)
        GMF_temp = GMF_model(hidden_size=hidden_dim, **kw_dict)
        recommender = NCF(factor_num=hidden_dim, num_layers=3, dropout=0.5, model= 'NeuMF-pre', GMF_model= GMF_temp, MLP_model=MLP_temp, **kw_dict)
    recommender_checkpoint = torch.load(Path(checkpoints_path, recommender_path))
    recommender.load_state_dict(recommender_checkpoint)
    recommender.eval()
    for param in recommender.parameters():
        param.requires_grad= False
    return recommender
    
recommender = load_recommender()

# LXR definition and loading

In [16]:
class Explainer(nn.Module):
    def __init__(self, user_size, item_size, hidden_size):
        super(Explainer, self).__init__()
        
        self.users_fc = nn.Linear(in_features = user_size, out_features=hidden_size).to(device)
        self.items_fc = nn.Linear(in_features = item_size, out_features=hidden_size).to(device)
        self.bottleneck = nn.Sequential(
            nn.Tanh(),
            nn.Linear(in_features = hidden_size*2, out_features=hidden_size).to(device),
            nn.Tanh(),
            nn.Linear(in_features = hidden_size, out_features=user_size).to(device),
            nn.Sigmoid()
        ).to(device)
        
        
    def forward(self, user_tensor, item_tensor):
        user_output = self.users_fc(user_tensor.float())
        item_output = self.items_fc(item_tensor.float())
        combined_output = torch.cat((user_output, item_output), dim=-1)
        expl_scores = self.bottleneck(combined_output).to(device)
        return expl_scores

In [17]:
explainer = Explainer(num_features, num_items, lxr_dim)
lxr_checkpoint = torch.load(Path(checkpoints_path, lxr_path))
explainer.load_state_dict(lxr_checkpoint)
explainer.eval()
for param in explainer.parameters():
    param.requires_grad= False


# Help functions

In [32]:
from ipynb.fs.defs.help_functions import *
importlib.reload(ipynb.fs.defs.help_functions)
from ipynb.fs.defs.help_functions import *

# Baselines functions

In [19]:
from ipynb.fs.defs.lime import distance_to_proximity, LimeBase, get_lime_args, gaussian_kernel, get_lire_args
importlib.reload(ipynb.fs.defs.lime)
from ipynb.fs.defs.lime import distance_to_proximity, LimeBase, get_lime_args, gaussian_kernel, get_lire_args

lime = LimeBase(distance_to_proximity)

In [51]:
#User based similarities using Jaccard
def find_jaccard_mask(x, item_id, user_based_Jaccard_sim):
    user_hist = x # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    item_jaccard_dict = {}
    for i,j in enumerate(user_hist>0):
        if j:
            if (i,item_id) in user_based_Jaccard_sim:
                item_jaccard_dict[i]=user_based_Jaccard_sim[(i,item_id)] # add Jaccard similarity between items
            else:
                item_jaccard_dict[i] = 0            

    return item_jaccard_dict

In [52]:
#Cosine based similarities between users and items
def find_cosine_mask(x, item_id, item_cosine):
    user_hist = x # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    item_cosine_dict = {}
    for i,j in enumerate(user_hist>0):
        if j:
            if (i,item_id) in item_cosine:
                item_cosine_dict[i]=item_cosine[(i,item_id)]
            else:
                item_cosine_dict[i]=0

    return item_cosine_dict

In [53]:
#popularity mask
def find_pop_mask(x, item_id):
    user_hist = torch.Tensor(x).to(device) # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    item_pop_dict = {}
    
    for i,j in enumerate(user_hist>0):
        if j:
            item_pop_dict[i]=pop_array[i] # add the pop of the item to the dictionary
            
    return item_pop_dict

In [54]:
def find_lime_mask(x, item_id, min_pert, max_pert, num_of_perturbations, kernel_func, feature_selection, recommender, num_samples=10, method = 'POS', **kw_dict):
    user_hist = x # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    lime.kernel_fn = kernel_func
    neighborhood_data, neighborhood_labels, distances, item_id = get_lime_args(user_hist, item_id, recommender, all_items_tensor, min_pert = min_pert, max_pert = max_pert, num_of_perturbations = num_of_perturbations, seed = item_id, **kw_dict)
    if method=='POS':
        most_pop_items  = lime.explain_instance_with_data(neighborhood_data, neighborhood_labels, distances, item_id, num_samples, feature_selection, pos_neg='POS')
    if method=='NEG':
        most_pop_items  = lime.explain_instance_with_data(neighborhood_data, neighborhood_labels, distances, item_id, num_samples, feature_selection ,pos_neg='NEG')
        
    return most_pop_items 

In [55]:
def find_lire_mask(x, item_id, num_of_perturbations, kernel_func, feature_selection, recommender, proba=0.1, method = 'POS', **kw_dict):
    
    user_hist = x # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    lime.kernel_fn = kernel_func

    
    neighborhood_data, neighborhood_labels, distances, item_id = get_lire_args(user_hist, item_id, recommender, all_items_tensor, train_array, num_of_perturbations = num_of_perturbations, seed = item_id, proba=0.1, **kw_dict)
    if method=='POS':
        most_pop_items  = lime.explain_instance_with_data(neighborhood_data, neighborhood_labels, distances, item_id, num_of_perturbations, feature_selection, pos_neg='POS')
    if method=='NEG':
        most_pop_items  = lime.explain_instance_with_data(neighborhood_data, neighborhood_labels, distances, item_id, num_of_perturbations, feature_selection ,pos_neg='NEG')
        
    return most_pop_items

In [56]:
def find_tf_idf_mask(x, item_id, tf_idf_sim, user_id):

    x = x.cpu().detach().numpy()
    x[item_id] = 0
    user_tf_idf_scores = tf_idf_sim[user_id].copy()
  
    return user_tf_idf_scores

In [57]:
def find_fia_mask(user_tensor, item_tensor, item_id, recommender):
    y_pred = recommender_run(user_tensor, recommender, item_tensor, item_id, **kw_dict).to(device)
    items_fia = {}
    user_hist = user_tensor.cpu().detach().numpy().astype(int)
    
    for i in range(num_items):
        if(user_hist[i] == 1):
            user_hist[i] = 0
            user_tensor = torch.FloatTensor(user_hist).to(device)
            y_pred_without_item = recommender_run(user_tensor, recommender, item_tensor, item_id, 'single', **kw_dict).to(device)
            infl_score = y_pred - y_pred_without_item
            items_fia[i] = infl_score
            user_hist[i] = 1

    return items_fia

In [58]:
def find_accent_mask(user_tensor, user_id, item_tensor, item_id, recommender_model, top_k):
   
    items_accent = defaultdict(float)
    factor = top_k - 1
    user_accent_hist = user_tensor.cpu().detach().numpy().astype(int)

    #Get topk items
    sorted_indices = list(get_top_k(user_tensor, user_tensor, recommender_model, **kw_dict).keys())
    
    if top_k == 1:
        # When k=1, return the index of the first maximum value
        top_k_indices = [sorted_indices[0]]
    else:
        top_k_indices = sorted_indices[:top_k]
   

    for iteration, item_k_id in enumerate(top_k_indices):

        # Set topk items to 0 in the user's history
        user_accent_hist[item_k_id] = 0
        user_tensor = torch.FloatTensor(user_accent_hist).to(device)
       
        item_vector = items_array[item_k_id]
        item_tensor = torch.FloatTensor(item_vector).to(device)
              
        # Check influence of the items in the history on this specific item in topk
        fia_dict = find_fia_mask(user_tensor, item_tensor, item_k_id, recommender_model)
         
        # Sum up all differences between influence on top1 and other topk values
        if not iteration:
            for key in fia_dict.keys():
                items_accent[key] *= factor
        else:
            for key in fia_dict.keys():
                items_accent[key] -= fia_dict[key]
       
    for key in items_accent.keys():
        items_accent[key] *= -1    

    return items_accent

In [59]:
def find_lxr_mask(x, item_tensor):
    
    user_hist = x
    mask = explainer(user_hist, item_tensor)
    x_masked = user_hist*mask
    item_sim_dict = {}
    for i,j in enumerate(x_masked>0):
        if j:
            item_sim_dict[i]=x_masked[i] 
        
    return item_sim_dict

# Evaluation help functions

In [67]:
def single_user_expl(user_vector, user_tensor, item_id, item_tensor, num_items, recommender_model, user_id = None, mask_type = None):
    user_hist_size = np.sum(user_vector)

    if mask_type == 'lime':
        POS_sim_items = find_lime_mask(user_vector, item_id, 50, 100, 150, distance_to_proximity,'highest_weights', recommender_model, num_samples=user_hist_size, **kw_dict)
        NEG_sim_items = find_lime_mask(user_vector, item_id, 50, 100, 150, distance_to_proximity,'highest_weights', recommender_model, num_samples=user_hist_size, method = 'NEG', **kw_dict)
    elif mask_type == 'lire':
        POS_sim_items = find_lire_mask(user_vector, item_id, 200, distance_to_proximity,'highest_weights', recommender_model,proba = 0.1, **kw_dict)
        NEG_sim_items = find_lire_mask(user_vector, item_id, 200, distance_to_proximity,'highest_weights', recommender_model,proba = 0.1, method = 'NEG', **kw_dict)
    else:
        if mask_type == 'jaccard':
            sim_items = find_jaccard_mask(user_tensor, item_id, jaccard_dict)
        elif mask_type == 'cosine':
            sim_items = find_cosine_mask(user_tensor, item_id, cosine_dict)
        elif mask_type == 'pop':
            sim_items = find_pop_mask(user_tensor, item_id)
        elif mask_type == 'tf_idf':
            sim_items = find_tf_idf_mask(user_tensor, item_id, tf_idf_dict, user_id)
        elif mask_type == 'shap':
            sim_items = find_shapley_mask(user_tensor, user_id, recommender_model, shap_values, item_to_cluster)
        elif mask_type == 'fia':
            sim_items = find_fia_mask(user_tensor, item_tensor, item_id, recommender_model) 
        elif mask_type == 'accent':
            sim_items = find_accent_mask(user_tensor, user_id, item_tensor, item_id, recommender_model, 5)
        elif mask_type == 'lxr':
            sim_items = find_lxr_mask(user_tensor, item_tensor)
        
        POS_sim_items  = list(sorted(sim_items.items(), key=lambda item: item[1],reverse=True))[0:user_hist_size]
        
    return POS_sim_items

In [64]:
def single_user_metrics(user_vector, user_tensor, item_id, item_tensor, num_of_bins, recommender_model, expl_dict, **kw_dict):
    POS_masked = user_tensor
    NEG_masked = user_tensor
    POS_masked[item_id]=0
    NEG_masked[item_id]=0
    user_hist_size = np.sum(user_vector)
    
    
    bins=[0]+[len(x) for x in np.array_split(np.arange(user_hist_size), num_of_bins, axis=0)]
    
    POS_at_1 = [0]*(len(bins))
    POS_at_5 = [0]*(len(bins))
    POS_at_10=[0]*(len(bins))
    POS_at_20=[0]*(len(bins))
    POS_at_50=[0]*(len(bins))
    POS_at_100=[0]*(len(bins))
    
    NEG_at_1 = [0]*(len(bins))
    NEG_at_5 = [0]*(len(bins))
    NEG_at_10 = [0]*(len(bins))
    NEG_at_20 = [0]*(len(bins))
    NEG_at_50 = [0]*(len(bins))
    NEG_at_100 = [0]*(len(bins))
    
    DEL = [0]*(len(bins))
    INS = [0]*(len(bins))
    
    rankA_at_1 = [0]*(len(bins))
    rankA_at_5 = [0]*(len(bins))
    rankA_at_10 = [0]*(len(bins))
    rankA_at_20 = [0]*(len(bins))
    rankA_at_50 = [0]*(len(bins))
    rankA_at_100 = [0]*(len(bins))
    
    rankB = [0]*(len(bins))
    NDCG = [0]*(len(bins))

    
    POS_sim_items = expl_dict
    NEG_sim_items  = list(sorted(dict(POS_sim_items).items(), key=lambda item: item[1],reverse=False))
    
    total_items=0
    for i in range(len(bins)):
        total_items += bins[i]
            
        POS_masked = torch.zeros_like(user_tensor, dtype=torch.float32, device=device)
        
        for j in POS_sim_items[:total_items]:
            POS_masked[j[0]] = 1
        POS_masked = user_tensor - POS_masked # remove the masked items from the user history

        NEG_masked = torch.zeros_like(user_tensor, dtype=torch.float32, device=device)
        for j in NEG_sim_items[:total_items]:
            NEG_masked[j[0]] = 1
        NEG_masked = user_tensor - NEG_masked # remove the masked items from the user history 
        
        POS_ranked_list = get_top_k(POS_masked, user_tensor, recommender_model, **kw_dict)
        
        if item_id in list(POS_ranked_list.keys()):
            POS_index = list(POS_ranked_list.keys()).index(item_id)+1
        else:
            POS_index = num_items
        NEG_index = get_index_in_the_list(NEG_masked, user_tensor, item_id, recommender_model, **kw_dict)+1

        # for pos:
        POS_at_1[i] = 1 if POS_index <=1 else 0
        POS_at_5[i] = 1 if POS_index <=5 else 0
        POS_at_10[i] = 1 if POS_index <=10 else 0
        POS_at_20[i] = 1 if POS_index <=20 else 0
        POS_at_50[i] = 1 if POS_index <=50 else 0
        POS_at_100[i] = 1 if POS_index <=100 else 0

        # for neg:
        NEG_at_1[i] = 1 if NEG_index <=1 else 0
        NEG_at_5[i] = 1 if NEG_index <=5 else 0
        NEG_at_10[i] = 1 if NEG_index <=10 else 0
        NEG_at_20[i] = 1 if NEG_index <=20 else 0
        NEG_at_50[i] = 1 if NEG_index <=50 else 0
        NEG_at_100[i] = 1 if NEG_index <=100 else 0

        # for del:
        DEL[i] = float(recommender_run(POS_masked, recommender_model, item_tensor, item_id, **kw_dict).detach().cpu().numpy())

        # for ins:
        INS[i] = float(recommender_run(user_tensor-POS_masked, recommender_model, item_tensor, item_id, **kw_dict).detach().cpu().numpy())

        # for rankA:
        rankA_at_1[i] = max(0, (1+1-POS_index)/1)
        rankA_at_5[i] = max(0, (5+1-POS_index)/5)
        rankA_at_10[i] = max(0, (10+1-POS_index)/10)
        rankA_at_20[i] = max(0, (20+1-POS_index)/20)
        rankA_at_50[i] = max(0, (50+1-POS_index)/50)
        rankA_at_100[i] = max(0, (100+1-POS_index)/100)

        # for rankB:
        rankB[i] = 1/POS_index

        #for NDCG:
        NDCG[i]= get_ndcg(list(POS_ranked_list.keys()),item_id, **kw_dict)
        
    res = [DEL, INS, rankB, NDCG, POS_at_1, POS_at_5, POS_at_10, POS_at_20, POS_at_50, POS_at_100,  NEG_at_1, NEG_at_5, NEG_at_10, NEG_at_20, NEG_at_50, NEG_at_100,  rankA_at_1, rankA_at_5, rankA_at_10, rankA_at_20, rankA_at_50, rankA_at_100]
    for i in range(len(res)):
        res[i] = np.array(res[i])
        
    return res

In [68]:
create_dictionaries = True

if create_dictionaries:
    import time
    recommender.eval()
    # Evaluate the model on the test set
    
    jaccard_expl_dict = {}
    cosine_expl_dict = {}
    pop_expl_dict = {}
    lime_expl_dict = {}
    tf_idf_expl_dict = {}
    lire_expl_dict = {}
    fia_expl_dict = {}
    accent_expl_dict = {}
    lxr_expl_dict = {}
    
#     shap_expl_dict = {}


    with torch.no_grad():
        for i in range(test_array.shape[0]):
            if i%500 == 0:
                print(i)
            start_time = time.time()
            user_vector = test_array[i]
            user_tensor = torch.FloatTensor(user_vector).to(device)
            user_id = int(test_data.index[i])

            item_id = int(get_user_recommended_item(user_tensor, recommender, **kw_dict).detach().cpu().numpy())
            item_vector =  items_array[item_id]
            item_tensor = torch.FloatTensor(item_vector).to(device)

            user_vector[item_id] = 0
            user_tensor[item_id] = 0

            jaccard_expl_dict[user_id] = single_user_expl(user_vector, user_tensor, item_id, item_tensor, num_items, recommender, mask_type= 'jaccard')
            cosine_expl_dict[user_id] = single_user_expl(user_vector, user_tensor, item_id, item_tensor, num_items, recommender, mask_type= 'cosine')
            pop_expl_dict[user_id] = single_user_expl(user_vector, user_tensor, item_id, item_tensor, num_items, recommender, mask_type= 'pop')
            lime_expl_dict[user_id] = single_user_expl(user_vector, user_tensor, item_id, item_tensor, num_items, recommender, mask_type= 'lime')
            tf_idf_expl_dict[user_id] = single_user_expl(user_vector, user_tensor, item_id, item_tensor, num_items, recommender, mask_type= 'tf_idf', user_id=user_id)
            lire_expl_dict[user_id] = single_user_expl(user_vector, user_tensor, item_id, item_tensor, num_items, recommender, mask_type= 'lire')
            fia_expl_dict[user_id] = single_user_expl(user_vector, user_tensor, item_id, item_tensor, num_items, recommender, mask_type= 'fia')
            accent_expl_dict[user_id] = single_user_expl(user_vector, user_tensor, item_id, item_tensor, num_items, recommender, mask_type= 'accent')
            lxr_expl_dict[user_id] = single_user_expl(user_vector, user_tensor, item_id, item_tensor, num_items, recommender, mask_type= 'lxr')
#             shap_expl_dict[user_id] = single_user_expl(user_vector, item_id, num_items, recommender, mask_type= 'shap',user_id = user_id)
            prev_time = time.time()
            print("User {}, total time: {:.2f}".format(i,prev_time - start_time))


        with open(Path(files_path,f'{recommender_name}_jaccard_expl_dict.pkl'), 'wb') as handle:
            pickle.dump(jaccard_expl_dict, handle)

        with open(Path(files_path,f'{recommender_name}_cosine_expl_dict.pkl'), 'wb') as handle:
            pickle.dump(cosine_expl_dict, handle)

        with open(Path(files_path,f'{recommender_name}_pop_expl_dict.pkl'), 'wb') as handle:
            pickle.dump(pop_expl_dict, handle)

        with open(Path(files_path,f'{recommender_name}_lime_expl_dict.pkl'), 'wb') as handle:
            pickle.dump(lime_expl_dict, handle)

        with open(Path(files_path,f'{recommender_name}_tf_idf_expl_dict.pkl'), 'wb') as handle:
            pickle.dump(tf_idf_expl_dict, handle)

        with open(Path(files_path,f'{recommender_name}_lire_expl_dict.pkl'), 'wb') as handle:
            pickle.dump(lire_expl_dict, handle)
    
        with open(Path(files_path,f'{recommender_name}_fia_expl_dict.pkl'), 'wb') as handle:
            pickle.dump(fia_expl_dict, handle)

        with open(Path(files_path,f'{recommender_name}_accent_expl_dict.pkl'), 'wb') as handle:
            pickle.dump(accent_expl_dict, handle) 
            
        with open(Path(files_path,f'{recommender_name}_lxr_expl_dict.pkl'), 'wb') as handle:
            pickle.dump(lxr_expl_dict, handle)

    #     with open(Path(files_path,f'{recommender_name}_shap_expl_dict.pkl'), 'wb') as handle:
    #         pickle.dump(shap_expl_dict, handle)


else:
        with open(Path(files_path,f'{recommender_name}_jaccard_expl_dict.pkl'), 'rb') as handle:
            jaccard_expl_dict = pickle.load(handle) 

        with open(Path(files_path,f'{recommender_name}_cosine_expl_dict.pkl'), 'rb') as handle:
            cosine_expl_dict = pickle.load(handle)

        with open(Path(files_path,f'{recommender_name}_pop_expl_dict.pkl'), 'rb') as handle:
            pop_expl_dict = pickle.load(handle)

        with open(Path(files_path,f'{recommender_name}_lime_expl_dict.pkl'), 'rb') as handle:
            lime_expl_dict = pickle.load(handle)

        with open(Path(files_path,f'{recommender_name}_tf_idf_expl_dict.pkl'), 'rb') as handle:
            tf_idf_expl_dict = pickle.load(handle)

        with open(Path(files_path,f'{recommender_name}_lire_expl_dict_200_clip.pkl'), 'rb') as handle:
            lire_expl_dict_200_clip = pickle.load(handle)

        with open(Path(files_path,f'{recommender_name}_fia_expl_dict.pkl'), 'rb') as handle:
            fia_expl_dict = pickle.load(handle)

        with open(Path(files_path,f'{recommender_name}_lxr_expl_dict.pkl'), 'rb') as handle:
            lxr_expl_dict = pickle.load(handle)

    #     with open(Path(files_path,f'{recommender_name}_shap_expl_dict.pkl'), 'rb') as handle:
    #         shap_expl_dict = pickle.load(handle)


0
User 0, total time: 1.80
User 1, total time: 1.15
User 2, total time: 1.02
User 3, total time: 1.06
User 4, total time: 1.56
User 5, total time: 1.14
User 6, total time: 1.11
User 7, total time: 1.11
User 8, total time: 1.17
User 9, total time: 1.15
User 10, total time: 1.11
User 11, total time: 1.14
User 12, total time: 1.41
User 13, total time: 1.41
User 14, total time: 1.04
User 15, total time: 1.00
User 16, total time: 1.26
User 17, total time: 1.15
User 18, total time: 1.01
User 19, total time: 1.02
User 20, total time: 1.05
User 21, total time: 1.20
User 22, total time: 1.44
User 23, total time: 1.04
User 24, total time: 1.23
User 25, total time: 1.14
User 26, total time: 1.32
User 27, total time: 1.11
User 28, total time: 1.59
User 29, total time: 1.02
User 30, total time: 1.09
User 31, total time: 1.27
User 32, total time: 1.05
User 33, total time: 1.03
User 34, total time: 1.51
User 35, total time: 1.21
User 36, total time: 1.04
User 37, total time: 1.66
User 38, total time:

User 308, total time: 1.16
User 309, total time: 1.14
User 310, total time: 1.04
User 311, total time: 1.12
User 312, total time: 1.66
User 313, total time: 1.12
User 314, total time: 1.00
User 315, total time: 1.06
User 316, total time: 1.07
User 317, total time: 1.61
User 318, total time: 1.25
User 319, total time: 1.09
User 320, total time: 1.03
User 321, total time: 1.06
User 322, total time: 1.00
User 323, total time: 1.45
User 324, total time: 1.00
User 325, total time: 1.13
User 326, total time: 1.04
User 327, total time: 0.94
User 328, total time: 1.05
User 329, total time: 0.98
User 330, total time: 1.39
User 331, total time: 1.17
User 332, total time: 1.17
User 333, total time: 1.03
User 334, total time: 1.26
User 335, total time: 1.12
User 336, total time: 1.01
User 337, total time: 1.27
User 338, total time: 1.00
User 339, total time: 1.15
User 340, total time: 1.20
User 341, total time: 1.16
User 342, total time: 1.15
User 343, total time: 1.08
User 344, total time: 1.12
U

User 612, total time: 1.09
User 613, total time: 1.15
User 614, total time: 1.21
User 615, total time: 1.02
User 616, total time: 1.72
User 617, total time: 1.04
User 618, total time: 1.12
User 619, total time: 1.20
User 620, total time: 1.03
User 621, total time: 1.15
User 622, total time: 1.05
User 623, total time: 1.01
User 624, total time: 1.50
User 625, total time: 1.05
User 626, total time: 1.43
User 627, total time: 1.62
User 628, total time: 1.04
User 629, total time: 1.39
User 630, total time: 1.12
User 631, total time: 1.09
User 632, total time: 1.22
User 633, total time: 1.07
User 634, total time: 1.07
User 635, total time: 1.53
User 636, total time: 1.10
User 637, total time: 1.01
User 638, total time: 1.57
User 639, total time: 1.09
User 640, total time: 1.42
User 641, total time: 1.15
User 642, total time: 0.99
User 643, total time: 1.88
User 644, total time: 1.21
User 645, total time: 1.13
User 646, total time: 1.02
User 647, total time: 1.16
User 648, total time: 1.13
U

User 916, total time: 1.19
User 917, total time: 1.09
User 918, total time: 1.55
User 919, total time: 1.16
User 920, total time: 1.11
User 921, total time: 1.14
User 922, total time: 1.12
User 923, total time: 1.30
User 924, total time: 1.01
User 925, total time: 1.08
User 926, total time: 1.09
User 927, total time: 1.32
User 928, total time: 1.05
User 929, total time: 1.05
User 930, total time: 1.13
User 931, total time: 1.07
User 932, total time: 1.01
User 933, total time: 1.03
User 934, total time: 1.17
User 935, total time: 1.00
User 936, total time: 1.38
User 937, total time: 1.00
User 938, total time: 1.52
User 939, total time: 1.13
User 940, total time: 1.35
User 941, total time: 1.09
User 942, total time: 1.09
User 943, total time: 1.25
User 944, total time: 1.23
User 945, total time: 1.16
User 946, total time: 1.24
User 947, total time: 1.31
User 948, total time: 1.39
User 949, total time: 1.38
User 950, total time: 1.28
User 951, total time: 1.11
User 952, total time: 1.09
U

In [72]:
def eval_one_expl_type(expl_name):
    print(f' ============ Start explaining by {expl_name} ============')
    with open(Path(files_path,f'{recommender_name}_{expl_name}_expl_dict.pkl'), 'rb') as handle:
        expl_dict = pickle.load(handle)
    recommender.eval()
    # Evaluate the model on the test set
    num_of_bins = 11


    users_DEL = np.zeros(num_of_bins)
    users_INS = np.zeros(num_of_bins)
    reciprocal = np.zeros(num_of_bins)
    NDCG = np.zeros(num_of_bins)
    POS_at_1 = np.zeros(num_of_bins)
    POS_at_5 = np.zeros(num_of_bins)
    POS_at_10 = np.zeros(num_of_bins)
    POS_at_20 = np.zeros(num_of_bins)
    POS_at_50 = np.zeros(num_of_bins)
    POS_at_100 = np.zeros(num_of_bins)
    NEG_at_1 = np.zeros(num_of_bins)
    NEG_at_5 = np.zeros(num_of_bins)
    NEG_at_10 = np.zeros(num_of_bins)
    NEG_at_20 = np.zeros(num_of_bins)
    NEG_at_50 = np.zeros(num_of_bins)
    NEG_at_100 = np.zeros(num_of_bins)
    rank_at_1 = np.zeros(num_of_bins)
    rank_at_5 = np.zeros(num_of_bins)
    rank_at_10 = np.zeros(num_of_bins)
    rank_at_20 = np.zeros(num_of_bins)
    rank_at_50 = np.zeros(num_of_bins)
    rank_at_100 = np.zeros(num_of_bins)

    num_of_bins=10


    with torch.no_grad():
        for i in range(test_array.shape[0]):
            start_time = time.time()
            user_vector = test_array[i]
            user_tensor = torch.FloatTensor(user_vector).to(device)
            user_id = int(test_data.index[i])

            item_id = int(get_user_recommended_item(user_tensor, recommender, **kw_dict).detach().cpu().numpy())
            item_vector =  items_array[item_id]
            item_tensor = torch.FloatTensor(item_vector).to(device)

            user_vector[item_id] = 0
            user_tensor[item_id] = 0

            user_expl = expl_dict[user_id]

            res = single_user_metrics(user_vector, user_tensor, item_id, item_tensor, num_of_bins, recommender, user_expl, **kw_dict)
            users_DEL += res[0]
            users_INS += res[1]
            reciprocal += res[2]
            NDCG += res[3]
            POS_at_1 += res[4]
            POS_at_5 += res[5]
            POS_at_10 += res[6]
            POS_at_20 += res[7]
            POS_at_50 += res[8]
            POS_at_100 += res[9]
            NEG_at_1 += res[10]
            NEG_at_5 += res[11]
            NEG_at_10 += res[12]
            NEG_at_20 += res[13]
            NEG_at_50 += res[14]
            NEG_at_100 += res[15]
            rank_at_1 += res[16]
            rank_at_5 += res[17]
            rank_at_10 += res[18]
            rank_at_20 += res[19]
            rank_at_50 += res[20]
            rank_at_100 += res[21]

            if i%500 == 0:
                prev_time = time.time()
                print("User {}, total time: {:.2f}".format(i,prev_time - start_time))

    a = i+1

    print(f'users_DEL_{expl_name}: ', np.mean(users_DEL[1:])/a)
    print(f'users_INS_{expl_name}: ', np.mean(users_INS[1:])/a)
    print(f'reciprocal_{expl_name}: ', np.mean(reciprocal[1:])/a)
    print(f'NDCG_{expl_name}: ', np.mean(NDCG[1:])/a)
    print(f'POS_at_1_{expl_name}: ', np.mean(POS_at_1[1:])/a)
    print(f'POS_at_5_{expl_name}: ', np.mean(POS_at_5[1:])/a)
    print(f'POS_at_10_{expl_name}: ', np.mean(POS_at_10[1:])/a)
    print(f'POS_at_20_{expl_name}: ', np.mean(POS_at_20[1:])/a)
    print(f'POS_at_50_{expl_name}: ', np.mean(POS_at_50[1:])/a)
    print(f'POS_at_100_{expl_name}: ', np.mean(POS_at_100[1:])/a)
    print(f'NEG_at_1_{expl_name}: ', np.mean(NEG_at_1[1:])/a)
    print(f'NEG_at_5_{expl_name}: ', np.mean(NEG_at_5[1:])/a)
    print(f'NEG_at_10_{expl_name}: ', np.mean(NEG_at_10[1:])/a)
    print(f'NEG_at_20_{expl_name}: ', np.mean(NEG_at_20[1:])/a)
    print(f'NEG_at_50_{expl_name}: ', np.mean(NEG_at_50[1:])/a)
    print(f'NEG_at_100_{expl_name}: ', np.mean(NEG_at_100[1:])/a)
    print(f'rank_at_1_{expl_name}: ', np.mean(rank_at_1[1:])/a)
    print(f'rank_at_5_{expl_name}: ', np.mean(rank_at_5[1:])/a)
    print(f'rank_at_10_{expl_name}: ', np.mean(rank_at_10[1:])/a)
    print(f'rank_at_20_{expl_name}: ', np.mean(rank_at_20[1:])/a)
    print(f'rank_at_50_{expl_name}: ', np.mean(rank_at_50[1:])/a)
    print(f'rank_at_100_{expl_name}: ', np.mean(rank_at_100[1:])/a)
    
    print(np.mean(users_DEL[1:])/a , np.mean(users_INS[1:])/a , np.mean(reciprocal[1:])/a , np.mean(NDCG[1:])/a , np.mean(POS_at_1[1:])/a , np.mean(NEG_at_1[1:])/a , np.mean(rank_at_1[1:])/a , np.mean(POS_at_5[1:])/a , np.mean(NEG_at_5[1:])/a , np.mean(rank_at_5[1:])/a , np.mean(POS_at_10[1:])/a , np.mean(NEG_at_10[1:])/a , np.mean(rank_at_10[1:])/a , np.mean(POS_at_20[1:])/a , np.mean(NEG_at_20[1:])/a , np.mean(rank_at_20[1:])/a , np.mean(POS_at_50[1:])/a , np.mean(NEG_at_50[1:])/a , np.mean(rank_at_50[1:])/a , np.mean(POS_at_100[1:])/a , np.mean(NEG_at_100[1:])/a , np.mean(rank_at_100[1:])/a)

In [76]:
expl_names_list = ['tf_idf','pop','lime','lire','fia','accent','lxr']

In [78]:
for expl_name in expl_names_list:
    eval_one_expl_type(expl_name)